In [1]:
from pathlib import Path
from openvino.runtime import Core
import cv2
import numpy as np
import sys
from pathlib import Path

sys.path.append("./engine")
#import engine.engine3js as engine
from engine.parse_poses import parse_poses

In [2]:
# directory where model will be downloaded
base_model_dir = "model"

# model name as named in Open Model Zoo
model_name = "human-pose-estimation-3d-0001"
# selected precision (FP32, FP16)
precision = "FP32"

BASE_MODEL_NAME = f"{base_model_dir}/public/{model_name}/{model_name}"
model_path = Path(BASE_MODEL_NAME).with_suffix(".pth")
onnx_path = Path(BASE_MODEL_NAME).with_suffix(".onnx")

ir_model_path = f"model/public/{model_name}/{precision}/{model_name}.xml"
model_weights_path = f"model/public/{model_name}/{precision}/{model_name}.bin"

if not model_path.exists():
    download_command = (
        f"omz_downloader " f"--name {model_name} " f"--output_dir {base_model_dir}"
    )
    ! $download_command

In [3]:
if not onnx_path.exists():
    convert_command = (
        f"omz_converter "
        f"--name {model_name} "
        f"--precisions {precision} "
        f"--download_dir {base_model_dir} "
        f"--output_dir {base_model_dir}"
    )
    ! $convert_command

In [4]:
# initialize inference engine
ie_core = Core()
# read the network and corresponding weights from file
model = ie_core.read_model(model=ir_model_path, weights=model_weights_path)
# load the model on the CPU (you can use GPU or MYRIAD as well)
compiled_model = ie_core.compile_model(model=model, device_name="CPU")
infer_request = compiled_model.create_infer_request()
input_tensor_name = model.inputs[0].get_any_name()

# get input and output names of nodes
input_layer = compiled_model.input(0)
output_layers = list(compiled_model.outputs)

In [5]:
cap = cv2.VideoCapture('0327.mp4')

In [6]:
body_edges = np.array(
    [
        [0, 1], 
        [0, 9], [9, 10], [10, 11],    # neck - r_shoulder - r_elbow - r_wrist
        [0, 3], [3, 4], [4, 5],       # neck - l_shoulder - l_elbow - l_wrist
        [1, 15], [15, 16],            # nose - l_eye - l_ear
        [1, 17], [17, 18],            # nose - r_eye - r_ear
        [0, 6], [6, 7], [7, 8],       # neck - l_hip - l_knee - l_ankle
        [0, 12], [12, 13], [13, 14],  # neck - r_hip - r_knee - r_ankle
    ]
)

focal_length = -1  # default
stride = 8
player = None
skeleton_set = None

In [7]:
poses = []
i = 0
offset = []
while cap.isOpened():
    # Read the next frame
    ret, frame = cap.read()
    if not ret:
        break

    input_image = cv2.resize(frame, (input_layer.shape[3], input_layer.shape[2]))
    input_image = input_image.transpose((2, 0, 1))  # change data layout from HWC to CHW
    input_image = input_image.reshape(input_layer.shape)  # reshape to input shape
    # run inference
    infer_request.infer({input_tensor_name: input_image})

    # A set of three inference results is obtained
    results = {
        name: infer_request.get_tensor(name).data[:]
        for name in {"features", "heatmaps", "pafs"}
    }
    # Get the results
    results = (results["features"][0], results["heatmaps"][0], results["pafs"][0])
    poses_3d, poses_2d = parse_poses(results, 1, stride, focal_length, True)
    if len(poses_3d) > 0:
                # From here, you can rotate the 3D point positions using the function "draw_poses",
                # or you can directly make the correct mapping below to properly display the object image on the screen
                poses_3d_copy = poses_3d.copy()
                x = poses_3d_copy[:, 0::4]
                y = poses_3d_copy[:, 1::4]
                z = poses_3d_copy[:, 2::4]
                poses_3d[:, 0::4], poses_3d[:, 1::4], poses_3d[:, 2::4] = (
                    -z + np.ones(poses_3d[:, 2::4].shape) * 200,
                    -y + np.ones(poses_3d[:, 2::4].shape) * 100,
                    -x,
                )

                poses_3d = poses_3d.reshape(poses_3d.shape[0], 19, -1)[:, :, 0:3]
                poses_3d = poses_3d.astype(float)
                if len(offset) == 0:
                    # Find the center of the skeleton
                    center = np.mean(poses_3d, axis=(0, 1))

                    # Compute the offset to move the skeleton to the origin
                    offset = -center

                # Apply the offset to all points of the skeleton
                poses_3d += offset

    for pose_3d in poses_3d:
        frame_pose = {
            "frame": i,
            "pose": {
                "neck": {"y":pose_3d[0][0], "z":pose_3d[0][1], "x":pose_3d[0][2]},
                "r_shoulder": {"y":pose_3d[9][0], "z":pose_3d[9][1], "x":pose_3d[9][2]},
                "r_elbow": {"y":pose_3d[10][0], "z":pose_3d[10][1], "x":pose_3d[10][2]},
                "r_wrist": {"y":pose_3d[11][0], "z":pose_3d[11][1], "x":pose_3d[11][2]},
                "l_shoulder": {"y":pose_3d[3][0], "z":pose_3d[3][1], "x":pose_3d[3][2]},
                "l_elbow": {"y":pose_3d[4][0], "z":pose_3d[4][1], "x":pose_3d[4][2]},
                "l_wrist": {"y":pose_3d[5][0], "z":pose_3d[5][1], "x":pose_3d[5][2]},
                "l_eye": {"y":pose_3d[15][0], "z":pose_3d[15][1], "x":pose_3d[15][2]},
                "l_ear": {"y":pose_3d[16][0], "z":pose_3d[16][1], "x":pose_3d[16][2]},
                "r_eye": {"y":pose_3d[17][0], "z":pose_3d[17][1], "x":pose_3d[17][2]},
                "r_ear": {"y":pose_3d[18][0], "z":pose_3d[18][1], "x":pose_3d[18][2]},
                "nose": {"y":pose_3d[1][0], "z":pose_3d[1][1], "x":pose_3d[1][2]},
                "l_hip": {"y":pose_3d[6][0], "z":pose_3d[6][1], "x":pose_3d[6][2]},
                "l_knee": {"y":pose_3d[7][0], "z":pose_3d[7][1], "x":pose_3d[7][2]},
                "l_ankle": {"y":pose_3d[8][0], "z":pose_3d[8][1], "x":pose_3d[8][2]},
                "r_hip": {"y":pose_3d[12][0], "z":pose_3d[12][1], "x":pose_3d[12][2]},
                "r_knee": {"y":pose_3d[13][0], "z":pose_3d[13][1], "x":pose_3d[13][2]},
                "r_ankle": {"y":pose_3d[14][0], "z":pose_3d[14][1], "x":pose_3d[14][2]}
            }
        }
        poses.append(frame_pose)
    i += 1
    if cv2.waitKey(1) == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

In [8]:
import json
with open("poses.json", "w") as f:
	json.dump(poses, f)

In [9]:
poses_file = []
with open("C:\\Users\\mievst\\Desktop\\masters\\pose_export\\poses.json", "r") as f:
    poses_file = json.loads(f.read())
poses_file[0]

{'frame': 0,
 'pose': {'neck': {'y': -3.4301187615645574,
   'z': 26.738197326660156,
   'x': -2.4005995047719857},
  'r_shoulder': {'y': -6.78200169613487,
   'z': 26.99492645263672,
   'x': 10.722807382282458},
  'r_elbow': {'y': 0.8295095343338801,
   'z': 11.432426452636719,
   'x': 25.990328286823473},
  'r_wrist': {'y': 17.70267847964638,
   'z': 21.21147918701172,
   'x': 12.698721383747301},
  'l_shoulder': {'y': -1.0552713494551824,
   'z': 26.593650817871094,
   'x': -17.40291264182643},
  'l_elbow': {'y': 10.36936549136513,
   'z': 7.322731018066406,
   'x': -23.152106787029066},
  'l_wrist': {'y': 28.553035535310443,
   'z': 19.34674835205078,
   'x': -8.225517298045911},
  'l_eye': {'y': 13.980403699372943,
   'z': 39.93053436279297,
   'x': -0.8614001525075814},
  'l_ear': {'y': 5.737727918122943,
   'z': 39.55748748779297,
   'x': -8.068892027202406},
  'r_eye': {'y': 14.102245130037005,
   'z': 42.219032287597656,
   'x': 4.376554940876208},
  'r_ear': {'y': 6.172786511